<a href="https://colab.research.google.com/github/NosenkoArtem/Categorical-Encoding/blob/master/%D0%A1%D0%B5%D0%BC%D0%B8%D0%BD%D0%B0%D1%80_%E2%84%964.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Семинар 4. Непрерывные диффузионные модели

 *Автор*: Александр Колесов

## 1. Мотивация

$\textbf{Вопрос:}$
какую задачу на протяжении семинара 2 (методы оценивания score-функций) мы пытались решить различными способами — от минимизации дивергенции Фишера до Denoising score matching?

Это была задача, в которой по выборке из расспределения $p(x)$ нужно было оценить $\nabla_{x}\log p(x)$. Сейчас пришло время раскрыть мотивацию: почему это так необходимо в диффузионных моделях?

$\textbf{Вопрос:}$
какой процесс производил зашумление данных, рассмотренное во время семинара 3, и каков его вид?

$\textbf{Мотивация:}$

- Зашумление данных в дискретной модели проходило при помощи уравнения $x_{t+1} = \sqrt{1-\beta_{t}}x_{t} + \sqrt{\beta_{t}}\epsilon , \quad \epsilon \sim \mathcal{N}(0,I).$

- Чуть ниже вы узнаете, что это стохастическое дифференциальное уравнение (СДУ), то есть такой процесс можно формально записать как $$dx_{t} = f(x_{t},t)dt + G(t)dw_{t}.$$

- Помните, во время семинара 3 (Дискретные диффузионные модели (DDPM)) мы сказали, что развернуть по времени ОДУ можно, а вот СДУ — нет. Поэтому матчим средние прямого и обратного процессов.

- А что если такой инструмент разворота существует? Это так — теорема Андерсона. Согласно этой теореме обратный процесс выглядит так:
$$ dx_{t} = [f(x_{t},t) - \frac{1}{2}G(t)G(t)^{T}\nabla_{x} \log p_{t}(x_{t})]dt + G(t)dw_{t}.  $$


$\textbf{Вопрос:}$
прямой процесс в диффузионных моделях обучаемый или нет?

Поскольку в диффузионных моделях прямой процесс необучаемый, $f(x_{t},t)$ и $G(t)$ мы выбираем сами, и они являются фиксированными. Тогда, наблюдая СДУ для обратного процесса, мы видим, что СДУ использует те же $f(x_{t},t)$ и $G(t)$, а единственный ингредиент, который отличает СДУ от прямого и обратного процессов, — это $\nabla_{x} \log p_{t}(x_{t}).$

$\textbf{Вопрос:}$
что стоит за $\nabla_{x} \log p_{t}(x_{t})$ с точки зрения процесса?

Итак, если мы научимся вычислять в каждый момент времени $\nabla_{x} \log p_{t}(x_{t})$, то сможем запускать обратное СДУ и генерировать картинки из шума. Как раз то, что нужно!

$\textbf{Вопрос к аудитории:}$
какими методами можно оценить $\nabla_{x} \log p_{t}(x_{t})$ ? В чем недостатки таких подходов?

![title](https://drive.google.com/uc?id=1CGFbtY2mCjlIY8pjvoGevfa_32d4b1dj)



Безусловно, для решения задачи оценки score-функции можно использовать:

- Implicit Score matching;
- Denosing Score matching;
- NCSN.

Однако рассмотрим один произвольный момент времени $t$ и выпишем задачу поиска score-функции:

$$ \int p_{t}(x_{t}) || s_{\theta}(x_{t},t)  - \nabla_{x}\log p_{t}(x_{t}) ||_{2}^{2} dx_{t} \to \min_{\theta}.$$

Сделаем это так же, как часто расписывали в семинаре 2:

$$ \mathbb{E}_{p_{t}(x_{t})}|| s_{\theta}(x,t)||^{2}_{2} - 2 \int < s_{\theta}(x,t),\nabla_{x} p_{t}(x_{t})>dx_{t}.$$


$\textbf{Вопрос:}$
как можно представить маргинальное распределение в любой момент времени $p_{t}(x_{t})$ через нулевой момент времени?

Обратите внимание на простой факт:
$$p_{t}(x_{t}) = \int p_{t}(x_{t}, x_{0})dx_{0} = \int q(x_{t}|x_{0})p_{0}(x_{0})dx_{0}.  $$

Тогда наше выражение перепишется так:

$$ \mathbb{E}||s_{\theta}(x,t)||^{2}_{2} - 2\int_{x_{t}} \int_{x_{0}} p_{0}(x_{0})<s_{\theta}(x,t),\nabla_{x}q(x_{t}|x_{0})>dx_{t}.$$

И теперь, если мы знаем $\nabla_{x}q(x_{t}|x_{0})$, то можем вычислить это выражение, а значит, посчитать задачу оптимизации.

$\textbf{Вопрос:}$
как в более простом виде, с учетом выкладок выше, будет выглядеть задача оптимизации и какой из методов семинара 2 она будет больше напоминать?

$$\mathbb{E}_{p_{t}(x_{t})} || s_{\theta}(x_{t},t)- \nabla_{x} \log q_{t}(x_{t}|x_{0})||^{2}_{2} \to \min_{\theta}.$$

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Subset

from torchvision.datasets import ImageFolder
import torchvision
from torchvision.datasets import MNIST
from torchvision.transforms import (
    Resize,
    Normalize,
    Compose,
    RandomHorizontalFlip,
    ToTensor,
    Lambda,
    CenterCrop
)


import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import wandb
import os
import math
import functools
import string

from ml_collections import ConfigDict
from typing import Optional, Union, Callable
from tqdm.auto import trange


import sys
sys.path.append("..")
from  ContDDPM.configs.default_cm_2_config import create_default_cm_2_config

### 1.1. Вывод непрерывного прямого диффузионного процесса из дискретного

Теперь построим связующий мостик между семинарами 3 и 4.


1. Рассмотрим простую дискретную схему:

$$x_{i} = (1 - \frac{\beta \Delta t}{2})x_{i-1} .$$

$\textbf{Наша цель}$ — записать непрерывный аналог данной дискретной схемы:

- $x_{i} = x(\frac{i}{N});$
- $\Delta t = \frac{1}{N};$
- $t \in \{0,\frac{1}{N},...,\frac{N-1}{N} \};$

$$ x(t + \Delta t) = (1 - \frac{\beta \Delta t}{2})x(t);$$

$$ \frac{ x(t + \Delta t) - x(t)}{\Delta t} = - \frac{\beta}{2}x(t).$$

Тогда в непрерывном случае при маленьких значениях $\Delta t$ процесс имеет вид

$$ \frac{dx(t)}{dt} = - \frac{\beta}{2}x(t).$$

2. Рассмотрим следующую дискретную схему:

$$x_{i} = x_{i-1} - \beta_{i-1}\nabla f(x_{i-1}). $$

Отсутствие $\Delta t $ мешает нам пока записать непрерывный аналог данного выражения, потому давайте введем его следующим образом:

$$ \beta_{i-1} = \beta(t)\Delta t.$$

Тогда перепишем:

$$ \frac{x(t+\Delta t) - x(t)}{\Delta t} = - \beta(t)\nabla f(x(t)).$$

И тогда непрерывная запись выглядит как

$$ \frac{dx(t)}{dt} =  - \beta(t)\nabla f(x(t)). $$

3. Рассмотрим один шаг дискретного DDPM между двумя произвольными соседними моментами времени $i-1$  и $i$:

$$ x_{i} = \sqrt{1 - \beta_{i}}x_{i-1} +\sqrt{\beta_{i}}\epsilon_{i-1}, \quad \epsilon_{i-1} \sim \mathcal{N}(0,I).$$

Поскольку нам нужно перейти от дискретного случая к непрерывному, понятие приращения времени должно фигурировать в $dt = \frac{1}{N}$, где $N$ — число шагов диффузионного процесса. Поэтому масштабируем расписание шума:

- Ранее: $\beta_{0} = 0.02, \beta_{N} = 1.$

- Сейчас: $\overline{\beta_{0}} = N \beta_{0} , ..., \overline{\beta_{N}} = N \beta_{N} \implies \{ \overline\beta_{i} =  N\beta_{i}\}_{i=0}^{N}.$

Такое перемасштабирование шума позволяет определить текущее значение $\beta_{i}$ шума как $\beta_{i} = \frac{\overline{\beta_{i}}} {N}.$

Тогда можем переписать выражение выше:

$$ x_{i} = \sqrt{1 - \frac{\overline{\beta_{i}}}{N}}x_{i-1} + \sqrt{\frac{\overline{\beta_{i}}}{N}}\epsilon_{i-1} . $$

И при стремлении $N \to \infty$ полагаем

- $dt = \frac{1}{N};$
- $\{\overline{\beta_{i}}\}_{i=1}^{N}$ становится функцией от времени $\beta(t);$
- $\beta(\frac{i}{N}) = \overline{\beta_{i}};$
- $x(\frac{i}{N}) = x_{i};$
- $\epsilon(\frac{i}{N}){i} = \epsilon_{i} .$

С учетом последнего перепишем крайнее выражение:

$$ x_{i} = \sqrt{1 -   \beta\left(\frac{i}{N}\right)dt}x_{i-1} + \sqrt{  \beta\left(\frac{i}{N}\right)dt}\epsilon_{i-1}.$$

Переходя от индексов к времени  $t = \frac{i}{N}$:

$$ x(t + dt) = \sqrt{1 -   \beta(t +dt)dt}x(t) + \sqrt{  \beta(t+dt)dt}\epsilon(t).$$

Используем формулу Тейлора для квадратичного корня, поскольку $\beta \in [0,1]:$

$$ x(t + dt) \approx (1 - \frac{1}{2}\beta(t+dt)dt)x(t)  + \sqrt{\beta(t+dt)dt}\epsilon(t),$$

$$ x(t + dt) - x(t) = dx \approx - \frac{1}{2}\beta(t)x(t)dt + \sqrt{\beta(t)dt}\epsilon(t) .$$

Тогда  $dw_{t} = \sqrt{dt}\epsilon(t),$

$$dx = -\frac{1}{2}\beta(t)x(t)dt + \sqrt{\beta(t)}dw_{t}.$$

![ChessUrl]( https://stable-diffusion-art.com/wp-content/uploads/2022/12/image-79.png "chess")  

In [ ]:
#TODO: Не забудь обьяснить про приращение Винеровского что это корень времени

### 1.2.  Теорема Андерсона

Теперь вы умеете определять $\textbf{прямой}$ диффузионный процесс в прямом направлении от данных к шуму. И наша $\textbf{главная задача}$, как и в предыдущем семинаре, в том, чтобы на основе прямого процесса построить процесс $\textbf{обратный}$. Он как раз и направлен на генерацию данных из шума.

Тогда мы задаемся вопросом:  $\textbf{Как, имея прямой процесс, построить обратный в непрерывном случае?}$

Ответ на этот вопрос и дает теорема Андерсона.

 $\textbf{Теорема Андерсона}$(1982).
Пусть прямой диффузионный процесс описывается уравнением
$$dx = f(x,t)dt + G(t)dw.$$
Тогда соответствующий ему обратный диффузионный процесс задается уравнением
$$dx = [f(x,t) - \frac{1}{2} G(t) G(t)^{T}\nabla_{x}\log p_{t}(x)]dt + G(t)dw.$$

Тогда, изучая форму обратного процесса, видим, что обратный процесс нам известен. Он имеет следующие особености:

- Шумовой терм обратного процесса полностью совпадает с прямым.
- Также знакомый дрифт прямого процесса $f(x,t)$ входит в дрифт обратного.
- Однако появился новый член $\nabla_{x}\log p_{t}(x).$

По форме записи мы уже прекрасно пониманием, что это выражение является $\textbf{score-функцией}$. И теперь наша основная задача состоит в том чтобы ее найти — это ровно то, что мы разобрали в мотивации.

### 1.3. Основы теории стохастических дифференциальных уравнений

#### 1.3.1. Понимание структуры вида и вывод СДУ


Как вы уже знаете, динамика Ланжевена тоже есть некоторое СДУ. Напомним, что на интуитивном уровне динамика Ланжевена представляет собой следующую конструкцию:

$$ dx_{t} = f_{t} + g_{t},$$

где $f_{t}$ — детерминистичная часть $\epsilon\nabla_{x}\log p(x)$, в то время как $g(t) = \sqrt{2\epsilon}z_{i}$ — шумовой терм.

$\textbf{Вопросы:}$

- За что отвечает детерминистичный терм динамики Ланжевена?
- На какую оптимизационную процедуру похожа динамика Ланжевена без шумового слагаемого?
- За что отвечает шумовое слагаемое в динамике Ланжевена?

![title](https://s.iimg.su/s/13/HN504VnoyJeiaWvpfjlcJAjBT1UfFf6LJz6zeun7.png)



В показанной выше интуитивной терминологии детерминистического слагаемого и шума СДУ может быть представлено (например, для двумерного диффузионного процессса в случае $D=2$) как

$$ \frac{dx_{t}}{dt} = f(x,t) + G(x,t)w(t).$$

И сразу определим, кто есть кто в этой записи. Поскольку процесс наш двумерный (для примера), значит, диффундирует по обеим координатам — как по оси $x$, так и по оси $y$. Тогда прежде всего разберемся с размерностями входящих в уравнение величин:

- $dx_{t} =  \begin{pmatrix}x^{(1)}_{t} -x^{(1)}_{t-1}\\x^{(2)}_{t} - x^{(2)}_{t-1}\end{pmatrix}$, то есть это $D$-мерный вектор ($D=2$);
- $dt$ — это тоже $D$-мерный вектор ($D=2$), что отражает приращение времени по каждой из компонент процесса;
- $f(x_{t},t)=\begin{pmatrix}f^{1}(x_{t},t)\\ f^{2}(x_{t},t\end{pmatrix}$ — это $D$-мерный вектор дрифта ($D=2$);
- $G(x_{t},t)$ — диффузионная матрица размера $D\times D$ ($D=2$);
- $w(t)=\begin{pmatrix}w_{t}^{(1)}\\ w_{t}^{(2)}\end{pmatrix}$ — это белый шум в момент времени $t$ по каждой из компонент, то есть это $D$-мерный вектор ($D=2$).

Теперь стоит понять, что собственно обозначают дрифт и диффузионная матрица процесса. Какой в них смысл, что они несут?

$\textbf{Вопросы для аудитории:}$

- В чем суть дрифта процесса $f(x_{t},t)$ ?
- В чем суть диффузионной матрицы  $G(x_{t},t)$ ?


$\textbf{Ответ}$: Если с диффузионным дрифтом все понятно (он показывает тренд, куда двигается процесс), то с сутью диффузионной матрицы дела обстоят несколько сложнее.

Как мы говорили ранее, наш диффузионный процесс обладает тем свойством , что у него каждая компонента диффундирует и, безусловно, существуют какие-то корреляции между процессами по этим компонентам. Как раз такие корреляции и описывает диффузионная матрица.

Например, в случае двумерного процесса диффузионная матрица

$$ G(x_{t},t) = \begin{pmatrix} dx^{(1)}_{t} dx^{(1)}_{t} &   dx^{(1)}_{t} dx^{(2)}_{t}\\
 dx^{(2)}_{t} dx^{(1)}_{t}& dx^{(2)}_{t} dx^{(2)}_{t}\end{pmatrix}. $$


Однако определенное выше якобы СДУ $\textbf{не является}$ ДУ в общепринятом смысле, поскольку теория ДУ не допускает $\textbf{разрывных}$ функций, таких как $w(t)$. **К чему приводит такая разрывность?**

Рассмотрим проинтегрированное нами СДУ за весь период времени от $t_{0}$ до ${t}$ :

$$ x(t) - x(0) = \int_{t_{0}}^{t}f(x(t),t)dt + \int_{t_{0}}^{t} G(x(t),t)w(t)dt.$$




![title](https://upload.wikimedia.org/wikipedia/commons/9/97/RiemannInt.png)

$\textbf{Вопросы}$
- Являются ли оба представленных интеграла римановыми?
- Что собой представляет интеграл Римана?

Поскольку функция $f$ непрерывна от $t_{0}$ до ${t}$, она и интегрируема по Риману.

А теперь давайте обратим внимание на второй интеграл, представляющий следующущю предельную сумму:

$$\int_{t_{0}}^{t} G(x(t),t)w(t)dt = \lim_{K\to \infty} \sum_{k=1}^{K} G(x(t^{*}_{k}),t^{*}_{k}) w(t^{*}_{k})(t_{k+1}-t_{k}). $$

$\textbf{Вопросы для аудитории}$
- Откуда взяты $t^{*}_{k}$?
- Когда интеграл Римана существует? Подсказка: вспомните про интегральные суммы — верхние и нижние.
- Почему второй интеграл не сходится в смысле Римана?

Чтобы обеспечить сходимость этого интеграла, можно перейти к интегралу Стильтеса, более подробно о котором можно почитать по ссылке https://en.wikipedia.org/wiki/Lebesgue–Stieltjes_integration

$\textbf{Самое главное}$, чтобы определить интеграл Стилтьеса, необходимо рассмотреть $w(t)dt$ как инкремент некоторого случайного процесса, коим и является винеровский процесс $dw(t)$. И таким образом мы переходим к новой записи диффузионного процесса:
    
$$ G(x(t),t)w(t)dt \to G(x(t),t)dw(t).$$

![title](https://upload.wikimedia.org/wikipedia/commons/thumb/7/79/Wiener-process-5traces.svg/1040px-Wiener-process-5traces.svg.png)

$\textbf{Винеровский процесс:}$

Винеровский процесс — это процесс независимых приращений $dw(t)$:

1. $dw(t) = w(t_{k+1}) - w(t_{k}) \sim \mathcal{N}(0,Q dt_{k})$  c $dt_{k} = t_{k+1} -t_{k}$.
2. $Qdt_{k}=  \begin{pmatrix}dw(t)^{(1)}dw(t)^{(1)} & dw(t)^{(1)}dw(t)^{(2)} \\
dw(t)^{(2)}dw(t)^{(1)} & dw(t)^{(2)}dw(t)^{(2)}\end{pmatrix}$ — матрица двумерного винеровского процесса.
3. Процесс начинается с $w(0) = 0$.
4. Инкременты являются независимыми случайными величинами.

$\textbf{Вопрос для аудитории:}$
- В чем физический смысл матрицы винеровского пооцесса?

Таким образом, мы пришли к финальному виду СДУ:
    
$$ dx(t) = f(x(t),t)dt  + G(x(t),t)dw(t).$$

#### Дополнение к данной главе

**Обратите внимание**, что интеграл Стилтьеса тоже расходится  при произвольном выборе точки в отрезке. Однако его заменяют похожим интегралом — интегралом Ито, который всегда выбирает нижнюю точку на отрезке. Интеграл Ито полностью решает проблему расходимости второго интеграла, однако объяснение этой темы выходит за пределы занятия.

$\textbf{Полезные сссылки:}$

- Интеграл Ито: https://ocw.mit.edu/courses/18-s096-topics-in-mathematics-with-applications-in-finance-fall-2013/ef2c66c8079ba656210ad1fd4a5e2fa8_MIT18_S096F13_lecnote18.pdf

- Интеграл Стратановича: https://www.degruyter.com/document/doi/10.1515/9783110741278-022/html

![ChessUrl]( https://stable-diffusion-art.com/wp-content/uploads/2022/12/image-79.png "chess")   

#### 1.3.2. Формула Ито

$\textbf{Важно:}$ пока держим в уме нашу главную мотивацию:
- Знать $\textbf{истинное}$ значение для $\nabla_{x}\log q(x(t)|x(0))$.
- На него обучать score-модель $s_{\theta}(x,t)$.
- Запускать обратное СДУ для генерации изображений.

Для ответа на первый вопрос осталось познакомиться с еще одним важным инструментом — $\textbf{формулой Ито}$.

Формула Ито утверждает:  если у вас есть некоторый случайный процесс на $x(t) = x_{t}$ и вы знаете, как это процесс выглядит (например, $ dx_{t} = f(x_{t},t)dt + G(x_{t},t)dw_{t}  $ ), то вы легко поймете, как выглядит процесс на любую  $\textbf{скалярную}$ функцию от этого процесса $\phi(x_{t})$:

$$ d\phi = \frac{\partial \phi}{\partial t}dt + \frac{\partial \phi}{\partial x}dx + \frac{1}{2}\frac{\partial^{2}\phi}{\partial x^{2}}dx^{2} = \frac{\partial \phi}{\partial t}dt +\sum_{i}\frac{\partial \phi}{\partial x_{i}}dx_{i} + \frac{1}{2}\sum_{i,j}\frac{\partial^{2}\phi}{\partial x_{i} \partial x_{j}}dx_{i}dx_{j} .$$

Эта формула выводится в соответствии с формулой Тейлора.  

$\textbf{Вопрос:}$ выведите по формуле Ито $ \phi(x(t)) = \frac{1}{2}x^{2}$.

$\textbf{Сравнение приращения процесса и времени:}$

- $dw(t) = \sqrt{dt}$

$\textbf{Правила малости:}$

Эти правила следуют из того, что мы рассмтрели формулу Тейлора до 2-го порядка малости:

- $dw(t) dt = 0$;
- $dt dw(t) = 0$;
- $dw(t) dw(t) = Qdt$.

In [ ]:
# Вывод dw(t)dw(t) = Qdt

#### 1.3.3. Формула Фоккера—Планка

![title](https://s.iimg.su/s/14/d8udvBiQra9TdzuH8eLeVBTKIFnRhWh5blZY98rq.png)

Уравнение Фоккера—Планка показывает, какому закону в каждый момент времени подчиняются маргинальные распределения процесса.

$\textbf{Внимание:}$ а ведь это ровно то уравнение, которое описывает нужную нам $q(x(t)|x(0))$, а значит, зная аналитическую форму такого распределения, можно:

- взять логарифм $\log q(x(t)|x(0))$;
- посчитать градиент $\nabla_{x} \log q(x(t)|x(0))$.

Давайте разберемся, откуда появляется уравнение Фоккера—Планка.

1. Рассматриваем СДУ для ($D=2$)-мерного случайного процесса:

$$ dx_{t} = f(x_{t},t)dt + G(x_{t},t)dw_{t}, \quad x(t_{0}) \sim p_{0}.$$

2. Выпишем Формулу Ито:

$$ d\phi = \frac{\partial \phi}{\partial t}dt + \sum_{i=1}\frac{\partial \phi}{\partial x_{i}}dx_{i} +
\frac{1}{2}\sum_{i,j} \frac{\partial^{2}\phi}{\partial x_{i}\partial x_{j}}dx_{i}dx_{j}.$$

Считая для простоты, что потенциал не зависит от времени, выразим ранее определенную запись для произведения компонент процесса через матрицы процесса и броуновского движения $G$ и $Q$:

$$ d\phi = \sum_{i=1}\frac{\partial \phi}{\partial x_{i}}f(x(t),t)_{i}dt + \sum_{i=1}\frac{\partial \phi}{\partial x_{i}}[G(x(t),t)dw(t)]_{i} + \frac{1}{2}\sum_{i,j}\frac{\partial^{2} \phi}{\partial x_{i}\partial x_{j}}[GQG^{T}]_{ij}dt.$$

3. Домножаем все на $dt$ и берем математическое ожидание:

$$ \frac{d\mathbb{E}\phi}{dt} = \sum_{i=1}\mathbb{E}[\frac{\partial \phi}{\partial x_{i}}f(x(t),t)]_{i} + 0 +
\frac{1}{2}\sum_{i,j}\mathbb{E}[\frac{\partial^{2} \phi}{\partial x_{i}\partial x_{j}}[GQG^{T}]_{ij}].$$

4. Дважды применяем формулу интегрирования по частям и получаем

$$ \frac{\partial p(x(t),t)}{\partial t} + \sum_{i}\frac{\partial}{\partial x_{i}}[f_{i}(x(t),t)p(x(t),t)]=
\frac{1}{2}\sum_{i,j}\frac{\partial^{2}}{\partial x_{i}\partial x_{j}} [[GQG^{T}]_{ij}p(x(t),t)] .$$

$\textbf{Вопросы в аудиторию:}$ почему зануляется математическое ожидание в пункте 3?

#### 1.3.4. Процессс Орнштейна—Уленбека

![title](https://upload.wikimedia.org/wikipedia/commons/thumb/6/60/Ornstein-Uhlenbeck-5traces.svg/1200px-Ornstein-Uhlenbeck-5traces.svg.png)

Снова вспомним мотивацию:
    
- Знать $\textbf{истинное}$ значение для $\nabla_{x}\log q(x(t)|x(0))$.
- На него обучать score-модель $s_{\theta}(x,t)$.
- Запускать обратное СДУ для генерации изображений.

Казалось бы, мы получили способ нахождения нужной плотности через решение уравнения Фоккера—Планка, но проблема состоит в том $\textbf{решать уравнение в частных производных сложно}$, а значит, нужно найти путь полегче.

$\textbf{Идея}$

1. Расссмотрим такой процесс, у которого маргинальное распределение всегда нормальное.
2. Найдем два первых момента распределений.

К счастью, такой процесс существует — процесс Орнштейна—Уленбека, дрифт которого является $\textbf{линейной}$ функцией по аргументу. И это ровно та причина, по которой многие годы ученые рассматривали диффузионные модели, в которых дрифт был линейной функцией, а не обучаемой нейросетью. Потому что маргинал $ \log q(x(t)|x(0))$ надо знать.

Пример процесса Орнштейна—Уленбека:

$$ dx(t) = \alpha x(t) dt + G(x(t),t)dw(t) .$$

Больше информации о свойствах этого процесса можно узнать по ссылке

https://www.maths.ed.ac.uk/~toh/Files/hypercontractivity.pdf




#### 1.3.5. Обыкновенные дифференциальные уравнения для нахождения моментов

![title](https://iimg.su/s/16/ol01PKmT5UOUQA6xnBAenBkrpwNka1L3LFVGL4Pz.png)

Мы подошли к этапу, когда нам известен закон каждого искомого маргинального распределения, но мы не знаем его моменты:

$$ q(x(t)|x(0)) = \mathcal{N}(x(t)| \mu(t) = ? , \Sigma(t) = ?). $$

Поиском $ \mu(t), \Sigma(t)$ мы сейчас и займемся.

$\textbf{Моменты маргинальных распределений}$


Выбираем линейный дрифт: $f(x_{t},t) = \alpha x_{t}$.

Выпишем Формулу Ито для такого дрифта:

$$ \frac{\mathbb{E}f(x_{t},t)}{dt} = \alpha \mathbb{E}f(x_{t},t).$$

Если обозначить среднее процесса через $m(t) =\mathbb{E}f(x_{t},t)$, то на среднее процесса получаем следующее стохастическое дифференциальное уравнение (обыкновенное дифференциальное уравнение Эйлера):

$$ \frac{dm(t)}{dt} = \alpha m(t).$$

$\textbf{Задание 1.}$ Определите моменты маргинальных распределений процесса Орнштейна—Улебенбека:
$$ dx = -\lambda x dt + d\beta, \quad x(0)=x_{0},$$
$\lambda > 0, \beta(t)$ — броуновское движение с диффузионной константой $q$.

$\textbf{Задание 2.}$ Определите моменты маргинальных распределений синусно-диффузионного процесса:
$$ dx = \sin(x)dt + d\beta(t)$$  с диффузионной константой $q$


Маргинальные рапсределения для нашего прямого процесса будут выглядеть так:


$$p_{0t}(x(t)|x(0)) = \mathcal{N}(x(t); e^{-\frac{1}{4}t^{2} (\overline{\beta_{max}} -\overline{\beta_{min}})  - \frac{1}{2}t\overline{\beta_{min}}}x(0),I - Ie^{-\frac{1}{2}t^{2}(\overline{\beta_{max}} - \overline{\beta_{min}})  - t\overline{\beta_{min}}} ).$$

![ChessUrl]( https://stable-diffusion-art.com/wp-content/uploads/2022/12/image-79.png "chess")   

In [ ]:
class VP_SDE:

    def __init__(self, config):
        """Construct a Variance Preserving SDE.

        Args:
          beta_min: value of beta(0)
          beta_max: value of beta(1)
          N: number of discretization steps
        """
        self.N = config.sde.N
        self.beta_0 = config.sde.beta_min
        self.beta_1 = config.sde.beta_max
        self._T = config.sde.T

    @property
    def T(self):
        """
        returns: terminal time [1]
        """
        return self._T


    def sde(self, x, t):
        """
        Calculate drift coeff. and diffusion coeff. in forward SDE

        input:
        - x
        - t

        returns: drift with size [B,C,H,W], diffuison with size [1]
        """
        beta_t = self.beta_0 + (self.beta_1 - self.beta_0) * t # linear law
        drift = -0.5 * beta_t[:, None, None, None] * x # [B,C,H,W]
        diffusion = torch.sqrt(beta_t) #[1]

        return drift, diffusion


    def marginal_prob(self, x_0, t):
        """
        Calculate marginal q(x_t|x_0)'s mean and std

        input:
        - x
        - t

        returns:
        """
        log_mean = - 0.5 * t * self.beta_0 - 0.25 * (t ** 2) * (self.beta_1 - self.beta_0)  # ??
        mean = torch.exp(log_mean[:, None, None, None]) * x_0 # [B,C,H,W]
        std = torch.sqrt(1 - torch.exp(log_mean * 2)) # ??

        return mean, std


    def marginal_std(self, t):
        """
        Calculate marginal q(x_t|x_0)'s std

        input:
        - x
        - t

        returns:
        """
        log_mean = - 0.5 * t * self.beta_0 - 0.25 * (t ** 2) * (self.beta_1 - self.beta_0)
        std = torch.sqrt(1 - torch.exp(log_mean * 2))

        return std


    def prior_sampling(self, shape):
        """

        """
        return torch.randn(*shape)

### 1.4. Обратный диффузионный процесс

![title](https://theacademic.com/wp-content/uploads/2023/09/reverse_diffusion.png)

Следующий класс как раз и определяет семплирование обратным диффузионным процессом. Здесь значение истинного $\nabla_{x}\log p_{t}(x)$ заменяется значением обученной нами score-функции $s_{\theta}^{*}(x,t)$:
    
$$ dx =  [f(x,t) - \frac{1}{2}G(t)G(t)^{T}s_{\theta}^{*}(x,t)]dt + G(t)dw_{t}. $$

Тогда итерационная схема обратного процесса выглядит так:

$$ x_{i} = x_{i+1} - f_{i+1}(x_{i+1}) + G_{i+1}G_{i+1}^{T}s^{*}_{\theta}(x_{i+1},i+1) + G_{i+1}z_{i+1} .$$

$\textbf{Вопрос к аудитории}:$
что представляет собой $z_{i+1} $?



In [ ]:
class RSDE:

    def __init__(self, vp_sde, ode_sampling):
        self.N = vp_sde.N
        self.ode_sampling = ode_sampling
        self.sde_fn = vp_sde.sde

    @property
    def T(self):
        return vp_sde._T


    def sde(self, x, t, score_fn, y=None):
        """
        Create the drift and diffusion functions for the reverse SDE/ODE.

        y is here for class-conditional generation through score SDE/ODE
        """

        """
        Calculate drift and diffusion for reverse SDE/ODE


        ode_sampling - True -> reverse ODE
        ode_sampling - False -> reverse SDE
        """
        drift, diffusion = self.sde_fn(x, t)
        score = score_fn(x, t) # получаем значение score-функции

        # (-1/2 beta_t * x_t - beta_t * score)
        drift = drift - diffusion[:, None, None, None] ** 2 * score
        return drift, diffusion


### 1.5. Численная схема решения ОДУ (СДУ)

![title](https://drive.google.com/uc?id=1CGFbtY2mCjlIY8pjvoGevfa_32d4b1dj)

Теперь вы умеете запускать СДУ в обратном времени. Разумеется, тут не все так просто, как с ОДУ, где мы просто интегрируем в обратном направлении то же уравнение. Отсюда следует логичный вопрос.

$\textbf{Вопрос для размышления:}$
А нет ли такого ОДУ, которое ведет себя, как наше СДУ?

То есть нет ли какого-то ОДУ, чьи фазовые траектории создают такие же маргинальные распределения, что и случайные траектории СДУ?

$\textbf{Вопросы:}$
- Зачем нужно ОДУ?
- Что бы мы хотели от такого СДУ?
- Как можно было бы прийти к идее создания такого СДУ?

Таким образом, если ОДУ имеет те же маргианльные распределения процесса, что и СДУ, то можно запускать ОДУ в прямом направлении, а потом просто интегрировать в обратном направлении, тем самым имитируя обратный процесс.

Займемся выводом такого ОДУ.

Снова рассмотрим наше СДУ для прямого процесса, чтобы вывести ОДУ, которое имеет те же маргиналы, что и это СДУ:

$$dx_{t} = f(x_{t},t)dt + G(x_{t},t)dw_{t} .$$

$\textbf{Вопросы:}$
- Дрифт — скалярная функция?
- Что такое матрица процесса?

Тогда запишем уравнение Фоккера—Планка:

$$\frac{\partial p_{t}(x)}{\partial t} = - \sum_{i=1}^{d} \frac{\partial}{\partial x_{i}} [f_{i}(x,t)p_{t}(x)] + \frac{1}{2}\sum_{i=1}^{d}\sum_{j=1}^{d}\frac{\partial^{2}}{\partial x_{i} \partial x_{j}}[\sum_{k=1}^{d}G_{ik}G_{jk}p_{t}(x)].$$

Перепишем данное уравнение:
$$\frac{\partial p_{t}(x)}{\partial t} = - \sum_{i=1}^{d} \frac{\partial}{\partial x_{i}} [f_{i}(x,t)p_{t}(x)] + \frac{1}{2}\sum_{i=1}^{d}\frac{\partial}{\partial x_{i} }[\sum_{j=1}^{d}\frac{\partial}{\partial x_{j} }[\sum_{k=1}^{d}G_{ik}G_{jk}p_{t}(x)]] $$

Распишем последнюю производную последнего выражения:

$$ \sum_{j=1}^{d}\frac{\partial}{\partial x_{j} }[\sum_{k=1}^{d}G_{ik}G_{jk}p_{t}(x)] = \sum_{j=1}^{d}\frac{\partial}{\partial x_{j} }[\sum_{k=1}^{d}G_{ik}G_{jk}]p_{t}(x) + \sum_{j=1}^{d}\sum_{k=1}^{d}G_{ik}G_{jk}[\frac{\partial}{\partial x_{j}}\log p_{t}(x)] = $$

$$ = p_{t}(x)\nabla_{x}[G(x,t)G(x,t)^{T}] + p_{t}(x)G(x,t)G(x,t)^{T}\nabla_{x}\log p_{t}(x) $$

Тогда в исходное уравнение Фоккера—Планка подставим итоговое выражение для посчитанной второй производной:


$$\frac{\partial p_{t}(x)}{\partial t} = - \sum_{i=1}^{d} \frac{\partial}{\partial x_{i}} [f_{i}(x,t)p_{t}(x)] +  \frac{1}{2}\sum_{i=1}^{d}\frac{\partial}{\partial x_{i}} [p_{t}(x)\nabla_{x}[G(x,t)G(x,t)^{T}] + p_{t}(x)G(x,t)G(x,t)^{T}\nabla_{x}\log p_{t}(x)].$$

Объединим два слагаемых в правой части в одно:

$$\frac{\partial p_{t}(x)}{\partial t} = - \sum_{i=1}^{d} \frac{\partial}{\partial x_{i}} \{ f_{i}(x,t)p_{t}(x) - \frac{1}{2}[\nabla_{x}[G(x,t)G(x,t)^{T}] +  G(x,t)G(x,t)^{T}\nabla_{x}\log p_{t}(x)]p_{t}(x)\}  = -\sum_{i=1}^{d}\frac{\partial}{\partial x_{i}}[\hat{f}_{i}(x,t)p_{t}(x)],$$

где через дрифт с шляпкой мы обозначили

$$\hat{f}(x,t) = f(x,t) - \frac{1}{2}\nabla_{x}[G(x,t)G(x,t)^{T}] - \frac{1}{2}G(x,t)G(x,t)^{T}\nabla_{x}\log p_{t}(x).$$

$\textbf{Вопрос к аудитории}:$
почему выше не описан второй терм из последней формулы в уравнении обратного диффузионного процесса?

Таким образом,  ОДУ $dx = \hat{f}(x,t)dt$ имеет те же маргинальные траектории, что и СДУ $dx = f(x,t)dt + G(x,t)dw_{t}$.

$\textbf{Вопрос:}$
чем различаются траектории ОДУ и СДУ?

Это ОДУ называется в литературе Probablility flow.

In [ ]:
class EulerDiffEqSolver:
    def __init__(self, sde, rsde, score_fn, ode_sampling = False):
        self.sde = sde
        self.score_fn = score_fn
        self.ode_sampling = ode_sampling
        self.rsde =  rsde

    def step(self, x, t, y=None):
        """
        Implement reverse SDE/ODE Euler solver
        """

        """
        x_mean = deterministic part
        x = x_mean + noise (yet another noise sampling)
        """

        dt = -1 / self.rsde.N
        z = torch.randn(x.shape).to(x.device)

        drift, diffusion = self.rsde.sde(x, t, self.score_fn)
        x_mean = x + drift * dt
        x = x_mean + np.sqrt(-dt) * diffusion[:, None, None, None] * z

        return x, x_mean

### 1.6. Оценка правдоподобия диффузионной модели

![title](https://uvadlc-notebooks.readthedocs.io/en/latest/_images/normalizing_flow_layout.png)

Остался один из главных вопросов: как нам оценить качество генерации диффузионных моделей?

$\textbf{Вопрос:}$
какие метрики качества вы бы предложили?

Можно оценить правдоподобие.

У нас есть ОДУ обратного процесса:

$$dx = \{ f(x,t) - \frac{1}{2}\nabla_{x}[G(x,t)G(x,t)^{T} - \frac{1}{2}G(x,t)G(x,t)^{T}s_{\theta}(x,t)\}dt,$$

$$ dx = \hat{f}(x,t)dt.$$

$\textbf{Вопрос:}$
ОДУ прямого процесса отличается от обратного ОДУ?

Согласно формуле замены переменной в кратном интеграле
$$\log p_{0}(x(0)) = \log p_{T}(x(T)) + \int_{0}^{T} \nabla_{x}\hat{f}(x,t)dt.$$

$\textbf{Вопрос:}$
оцениваем мы правдоподобие на инференсе или на трейне?


Больше про нормализационные потоки и детерминистичные функции преобразования можно посмотреть здесь:
 https://pytorch-lighting.readthedocs.io/en/latest/notebooks/course_UvA-DL/09-normalizing-flows.html

## 2.  Прямой диффузионный процесс

![ChessUrl]( https://stable-diffusion-art.com/wp-content/uploads/2022/12/image-79.png "chess")   

Маргинальные распределения для нашего прямого процесса будут выглядеть так:


$$p_{0t}(x(t)|x(0)) = \mathcal{N}(x(t); e^{-\frac{1}{4}t^{2} (\overline{\beta_{max}} -\overline{\beta_{min}})  - \frac{1}{2}t\overline{\beta_{min}}}x(0),I - Ie^{-\frac{1}{2}t^{2}(\overline{\beta_{max}} - \overline{\beta_{min}})  - t\overline{\beta_{min}}} ).$$

In [ ]:
class VP_SDE:

    def __init__(self, config):
        """Construct a Variance Preserving SDE.

        Args:
          beta_min: value of beta(0)
          beta_max: value of beta(1)
          N: number of discretization steps
        """
        self.N = config.sde.N
        self.beta_0 = config.sde.beta_min
        self.beta_1 = config.sde.beta_max
        self._T = config.sde.T

    @property
    def T(self):
        """
        returns: terminal time [1]
        """
        return self._T


    def sde(self, x, t):
        """
        Calculate drift coeff. and diffusion coeff. in forward SDE

        input:
        - x
        - t

        returns: drift with size [B,C,H,W], diffuison with size [1]
        """
        beta_t = self.beta_0 + (self.beta_1 - self.beta_0) * t # linear law
        drift = -0.5 * beta_t[:, None, None, None] * x # [B,C,H,W]
        diffusion = torch.sqrt(beta_t) #[1]

        return drift, diffusion


    def marginal_prob(self, x_0, t):
        """
        Calculate marginal q(x_t|x_0)'s mean and std

        input:
        - x
        - t

        returns:
        """
        log_mean = - 0.5 * t * self.beta_0 - 0.25 * (t ** 2) * (self.beta_1 - self.beta_0)  # ??
        mean = torch.exp(log_mean[:, None, None, None]) * x_0 # [B,C,H,W]
        std = torch.sqrt(1 - torch.exp(log_mean * 2)) # ??

        return mean, std


    def marginal_std(self, t):
        """
        Calculate marginal q(x_t|x_0)'s std

        input:
        - x
        - t

        returns:
        """
        log_mean = - 0.5 * t * self.beta_0 - 0.25 * (t ** 2) * (self.beta_1 - self.beta_0)
        std = torch.sqrt(1 - torch.exp(log_mean * 2))

        return std


    def prior_sampling(self, shape):
        """

        """
        return torch.randn(*shape)

## 3. Обратный диффузионный процесс

![title](https://theacademic.com/wp-content/uploads/2023/09/reverse_diffusion.png)

Следующий класс как раз и определяет семплирование обратным диффузионным процессом. Здесь значение истинного $\nabla_{x}\log p_{t}(x)$ заменяется значением обученной нами score-функции $s_{\theta}^{*}(x,t)$:
    
$$ dx =  [f(x,t) - \frac{1}{2}G(t)G(t)^{T}s_{\theta}^{*}(x,t)]dt + G(t)dw_{t} .$$

Тогда итерационная схема обратного процесса выглядит так:

$$ x_{i} = x_{i+1} - f_{i+1}(x_{i+1}) + G_{i+1}G_{i+1}^{T}s^{*}_{\theta}(x_{i+1},i+1) + G_{i+1}z_{i+1} .$$




In [ ]:
class RSDE:

    def __init__(self, vp_sde, ode_sampling):
        self.N = vp_sde.N
        self.ode_sampling = ode_sampling
        self.sde_fn = vp_sde.sde

    @property
    def T(self):
        return vp_sde._T


    def sde(self, x, t, score_fn, y=None):
        """
        Create the drift and diffusion functions for the reverse SDE/ODE.

        y is here for class-conditional generation through score SDE/ODE
        """

        """
        Calculate drift and diffusion for reverse SDE/ODE


        ode_sampling - True -> reverse ODE
        ode_sampling - False -> reverse SDE
        """
        drift, diffusion = self.sde_fn(x, t)
        score = score_fn(x, t) # получаем значение score-функции

        # (-1/2 beta_t * x_t - beta_t * score)
        drift = drift - diffusion[:, None, None, None] ** 2 * score
        return drift, diffusion


## 4. Численная схема решения ОДУ (СДУ)

![title](https://drive.google.com/uc?id=1CGFbtY2mCjlIY8pjvoGevfa_32d4b1dj)

Теперь вы умеете запускать СДУ в обратном времени. Разумеется, тут не все так просто, как с ОДУ, где мы просто интегрируем в обратном направлении то же уравнение. Отсюда логичный вопрос:

$\textbf{Вопрос для размышления:}$
Нет ли такого ОДУ, который ведет себя, как наше СДУ?

То есть нет ли какого-то ОДУ, чьи фазовые траектории создают такие же маргинальные распределения, что и случайные траектории СДУ?

$\textbf{Вопросы:}$
- Зачем нужно ОДУ?
- Что бы мы хотели от такого СДУ?
- Как можно было бы прийти к идее создания такого СДУ?

Таким образом, если ОДУ имеет те же маргинальные распределения процесса, что и СДУ, то можно просто запускать ОДУ в прямом направлении, а потом интегрировать в обратном направлении, тем самым имитируя обратный процесс.

Займемся выводом такого ОДУ.

Снова рассмотрим наше СДУ для прямого процесса, чтобы вывести ОДУ, которое имеет те же маргиналы, что и это СДУ:

$$dx_{t} = f(x_{t},t)dt + G(x_{t},t)dw_{t} $$

$\textbf{Вопросы:}$
- Дрифт — скалярная функция?
- Что такое матрица процесса?

Тогда запишем уравнение Фоккера—Планка:

$$\frac{\partial p_{t}(x)}{\partial t} = - \sum_{i=1}^{d} \frac{\partial}{\partial x_{i}} [f_{i}(x,t)p_{t}(x)] + \frac{1}{2}\sum_{i=1}^{d}\sum_{j=1}^{d}\frac{\partial^{2}}{\partial x_{i} \partial x_{j}}[\sum_{k=1}^{d}G_{ik}G_{jk}p_{t}(x)]$$

Перепишем данное уравнение:
$$\frac{\partial p_{t}(x)}{\partial t} = - \sum_{i=1}^{d} \frac{\partial}{\partial x_{i}} [f_{i}(x,t)p_{t}(x)] + \frac{1}{2}\sum_{i=1}^{d}\frac{\partial}{\partial x_{i} }[\sum_{j=1}^{d}\frac{\partial}{\partial x_{j} }[\sum_{k=1}^{d}G_{ik}G_{jk}p_{t}(x)]] $$

Распишем последнюю производную последнего выражения:

$$ \sum_{j=1}^{d}\frac{\partial}{\partial x_{j} }[\sum_{k=1}^{d}G_{ik}G_{jk}p_{t}(x)] = \sum_{j=1}^{d}\frac{\partial}{\partial x_{j} }[\sum_{k=1}^{d}G_{ik}G_{jk}]p_{t}(x) + \sum_{j=1}^{d}\sum_{k=1}^{d}G_{ik}G_{jk}[\frac{\partial}{\partial x_{j}}\log p_{t}(x)] = $$

$$ = p_{t}(x)\nabla_{x}[G(x,t)G(x,t)^{T}] + p_{t}(x)G(x,t)G(x,t)^{T}\nabla_{x}\log p_{t}(x) $$

Тогда в исходного Фоккера—Планка подставим итоговое выражение для посчитанной второй производной:


$$\frac{\partial p_{t}(x)}{\partial t} = - \sum_{i=1}^{d} \frac{\partial}{\partial x_{i}} [f_{i}(x,t)p_{t}(x)] +  \frac{1}{2}\sum_{i=1}^{d}\frac{\partial}{\partial x_{i}} [p_{t}(x)\nabla_{x}[G(x,t)G(x,t)^{T}] + p_{t}(x)G(x,t)G(x,t)^{T}\nabla_{x}\log p_{t}(x)]$$

Объединим два слагаемых в правой части в одно:

$$\frac{\partial p_{t}(x)}{\partial t} = - \sum_{i=1}^{d} \frac{\partial}{\partial x_{i}} \{ f_{i}(x,t)p_{t}(x) - \frac{1}{2}[\nabla_{x}[G(x,t)G(x,t)^{T}] +  G(x,t)G(x,t)^{T}\nabla_{x}\log p_{t}(x)]p_{t}(x)\}  = -\sum_{i=1}^{d}\frac{\partial}{\partial x_{i}}[\hat{f}_{i}(x,t)p_{t}(x)]$$

Здесь через дрифт с шляпкой мы обозначили функцию

$$\hat{f}(x,t) = f(x,t) - \frac{1}{2}\nabla_{x}[G(x,t)G(x,t)^{T}] - \frac{1}{2}G(x,t)G(x,t)^{T}\nabla_{x}\log p_{t}(x)$$

$\textbf{Вопрос к аудитории.}$
Почему выше не описан второй терм из последней формулы в уравнении обратного диффузионного процесса?

Таким образом,  ОДУ $dx = \hat{f}(x,t)dt$ имеет те же маргинальные траектории, что и СДУ $dx = f(x,t)dt + G(x,t)dw_{t}$.

$\textbf{Вопрос:}$
чем различаются траектории ОДУ и СДУ?

Это ОДУ называется в литературе Probablility flow.

In [ ]:
class EulerDiffEqSolver:
    def __init__(self, sde, rsde, score_fn, ode_sampling = False):
        self.sde = sde
        self.score_fn = score_fn
        self.ode_sampling = ode_sampling
        self.rsde =  rsde

    def step(self, x, t, y=None):
        """
        Implement reverse SDE/ODE Euler solver
        """

        """
        x_mean = deterministic part
        x = x_mean + noise (yet another noise sampling)
        """

        dt = -1 / self.rsde.N
        z = torch.randn(x.shape).to(x.device)

        drift, diffusion = self.rsde.sde(x, t, self.score_fn)
        x_mean = x + drift * dt
        x = x_mean + np.sqrt(-dt) * diffusion[:, None, None, None] * z

        return x, x_mean

## 5. Модель глубокого обучения для DDPM

Эта часть представляет собой описание и имплементацию деталей модели глубокого обучения для работы с изображениями, а также некоторых оптимизационных инструментов, обеспечивающих значимый прирост качества на практике.

### 5.1. Экспоненциальное скользящее среднее

Источники:
- https://jcsustem.com/blog/benefits-of-using-ema-in-deep-learning

- https://shallbd.com/understanding-ema-in-machine-learning-everything-you-need-to-know/

Что такое экспоненциальное скользящее среднее (ЕМА)?

EMA позволяет обновлять веса модели глубокого обучения путем усреднения предыдущих значений, придавая большую важность последним. Применение данного инструмента для моделей несет определенную значимость, поскольку, давая большую значимость последним значениям весов, ЕМА может охватить тренд оптимизации весов, делая модель более робастной.

ЕМА работает следующим образом:

$$ \theta_{ema} = \lambda\theta_{current} + (1-\lambda)\theta_{ema} .$$

Преимущества использования ЕМА:
    
- 1. Стабилизирует модель обучения за счет сглаживания колебания градиентов.
- 2. Уменьшает шум при обновлении весов модели.
- 3. Обеспечивает ускорение сходимости.
- 4. Обеспечивает большую устойчивость модели по отношению к гиперпараметрам.


Ниже преставлен класс для ЕМА.
    
1.  Инициализация

Важно понимать, что мы определяем фактор сглаживания колебаний градиентов модели, а также инициализируем ЕМА параметрами модели (то есть текущим скользящим средним). Такое текущее скользящее среднеее мы сохраняем в переменную **shadow_params**.

2. Копирование

Эта операция позволяет инициализировать параметры модели при помощи текущего скользящего среднего.

3. Сохранение

Следующий метод позволяет нам сохранять все текущие параметры модели.

4. Словарь (загрузка/разгрузка)

Позволяет сохранять текущий фактор сглаживания и текущее скользящее среднее.

5. Обновление

Мы модифицируем переменную **shadow_params**, в которой хранятся значения текущего скользящего среднего, и обновляем его с учетом прошлого скользящего среднего и крайних весов модели.

In [ ]:
class ExponentialMovingAverage:

    """
    Maintains (exponential) moving average of a set of parameters.
    """

    def __init__(self, parameters, decay, use_num_updates=True):
        """
        Args:
          parameters: Iterable of `torch.nn.Parameter`; usually the result of
            `model.parameters()`.
          decay: The exponential decay.
          use_num_updates: Whether to use number of updates when computing
            averages.
        """
        if decay < 0.0 or decay > 1.0:
            raise ValueError('Decay must be between 0 and 1')
        self.decay = decay
        self.num_updates = 0 if use_num_updates else None
        self.shadow_params = [p.clone().detach()
                              for p in parameters if p.requires_grad]
        self.collected_params = []

    def copy_to(self, parameters):
        """
        Copy current parameters into given collection of parameters.

        Args:
        parameters: Iterable of `torch.nn.Parameter`; the parameters to be
        updated with the stored moving averages.
        """
        parameters = [p for p in parameters if p.requires_grad]
        for s_param, param in zip(self.shadow_params, parameters):
            if param.requires_grad:
                param.data.copy_(s_param.data)

    def store(self, parameters):
        """
        Save the current parameters for restoring later.

        Args:
          parameters: Iterable of `torch.nn.Parameter`; the parameters to be
            temporarily stored.
        """
        self.collected_params = [param.clone() for param in parameters]


    def state_dict(self):
        return dict(decay=self.decay, num_updates=self.num_updates,
                    shadow_params=self.shadow_params)

    def load_state_dict(self, state_dict):
        self.decay = state_dict['decay']
        self.num_updates = state_dict['num_updates']
        self.shadow_params = state_dict['shadow_params']


    def update(self, parameters):
        """
        Update currently maintained parameters.

        Call this every time the parameters are updated, such as the result of
        the `optimizer.step()` call.

        Args:
          parameters: Iterable of `torch.nn.Parameter`; usually the same set of
            parameters used to initialize this object.
        """
        decay = self.decay
        if self.num_updates is not None:
            self.num_updates += 1
            decay = min(decay, (1 + self.num_updates) / (10 + self.num_updates))
        one_minus_decay = 1.0 - decay
        with torch.no_grad():
            parameters = [p for p in parameters if p.requires_grad]
            for s_param, param in zip(self.shadow_params, parameters):
                s_param.sub_(one_minus_decay * (s_param - param))


    def restore(self, parameters):
        """
        Restore the parameters stored with the `store` method.
        Useful to validate the model with EMA parameters without affecting the
        original optimization process. Store the parameters before the
        `copy_to` method. After validation (or model saving), use this to
        restore the former parameters.

        Args:
          parameters: Iterable of `torch.nn.Parameter`; the parameters to be
            updated with the stored parameters.
        """
        for c_param, param in zip(self.collected_params, parameters):
            param.data.copy_(c_param.data)



### 5.2. Функции активации

Теперь определим функцию активации для нашей архитектуры диффузионной модели.

In [ ]:
def get_act(config=None, act_str=None):
    """Get activation functions from the config file."""
    assert (config or act_str) is not None
    if config is not None:
        act_str = config.model.nonlinearity.lower()
    if act_str == 'elu':
        return nn.ELU()
    elif act_str == 'relu':
        return nn.ReLU()
    elif act_str == 'lrelu':
        return nn.LeakyReLU(negative_slope=0.2)
    elif act_str == 'swish':
        return nn.SiLU()
    else:
        raise NotImplementedError('activation function does not exist!')

### 5.3. Эмбеддинги условия (времени)

Можно включить информацию о времени с помощью гауссовских случайных функций (https://arxiv.org/abs/2006.10739). В частности, мы сначала пробуем $\omega \sim \mathcal{N}(\mathbf{0}, s ^ 2\mathbf {I})$, который затем фиксируется для модели (не поддается обучению). Для временного шага $t$ соответствующая гауссова случайная функция определяется как
\begin{align}
 [\sin(2\pi \omega t) ; \cos(2\pi \omega t)],
\end{align}
где $[\vec{a} ; \vec{b}]$ обозначает объединение векторов $\vec{a}$ и $\vec{b}$. Эта гауссова случайная функция может быть использована в качестве кодировки для временного шага $t$, чтобы сеть score могла определять значение $t$, используя эту кодировку. Мы увидим это далее в коде.

In [ ]:
def get_timestep_embedding(timesteps, embedding_dim, max_positions=10000):
    assert len(timesteps.shape) == 1  # and timesteps.dtype == tf.int32
    half_dim = embedding_dim // 2
    emb = math.log(max_positions) / (half_dim - 1)
    emb = torch.exp(torch.arange(half_dim, dtype=torch.float32, device=timesteps.device) * -emb)
    emb = timesteps.float()[:, None] * emb[None, :]
    emb = torch.cat([torch.sin(emb), torch.cos(emb)], dim=1)
    if embedding_dim % 2 == 1:  # zero pad
        emb = F.pad(emb, (0, 1), mode='constant')
    assert emb.shape == (timesteps.shape[0], embedding_dim)
    return emb

### 5.4. Инициализация параметров моделей

Зададим функцию, которая определяет начальную инициализацию весов нейросетевой модели.

In [ ]:
def variance_scaling(scale, mode, distribution,
                     in_axis=1, out_axis=0,
                     dtype=torch.float32,
                     device='cpu'):
    """Ported from JAX. """

    def _compute_fans(shape, in_axis=1, out_axis=0):
        receptive_field_size = np.prod(shape) / shape[in_axis] / shape[out_axis]
        fan_in = shape[in_axis] * receptive_field_size
        fan_out = shape[out_axis] * receptive_field_size
        return fan_in, fan_out

    def init(shape, dtype=dtype, device=device):
        fan_in, fan_out = _compute_fans(shape, in_axis, out_axis)
        if mode == "fan_in":
            denominator = fan_in
        elif mode == "fan_out":
            denominator = fan_out
        elif mode == "fan_avg":
            denominator = (fan_in + fan_out) / 2
        else:
            raise ValueError(
        "invalid mode for variance scaling initializer: {}".format(mode))
        variance = scale / denominator
        if distribution == "normal":
            return torch.randn(*shape, dtype=dtype, device=device) * np.sqrt(variance)
        elif distribution == "uniform":
            return (torch.rand(*shape, dtype=dtype, device=device) * 2. - 1.) * np.sqrt(3 * variance)
        else:
            raise ValueError("invalid distribution for variance scaling initializer")

    return init

In [ ]:
def default_init(scale=1.):
    """The same initialization used in DDPM."""
    scale = 1e-10 if scale == 0 else scale
    return variance_scaling(scale, 'fan_avg' , 'uniform' )

### 5.5. Сверточная нейронная сеть

Следующая функция возвращает сверточную нейронную сеть:

- https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html
- https://d2l.ai/chapter_convolutional-neural-networks/padding-and-strides.html


- размер ядер 3 на 3
- размер пэддингов (вставки по углам) 1
- размер страйдов (величина сдвига ядра) 1
- дилэйшен

In [ ]:
def ddpm_conv3x3(in_planes, out_planes, stride=1, bias=True, dilation=1, init_scale=1., padding=1):
    """3x3 convolution with DDPM initialization."""
    conv = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=padding,
                   dilation=dilation, bias=bias)
    conv.weight.data = default_init(init_scale)(conv.weight.data.shape)
    nn.init.zeros_(conv.bias)
    return conv

### 5.6. Механизм внимания

Сверточный модуль внимания (англ. сonvolutional block attention module) — простой, но эффективный модуль внимания для сверточных нейросетей. Применяется для задач детектирования обьектов на изображениях и классификации с входными данными больших размерностей. Данный модуль внимания состоит из двух последовательно применяемых подмодулей — канального (применяется ко всем каналам одного пикселя с изображения) и пространственного (применяется ко всему изображению с фиксированным каналом).

Напоминаем о том, что есть механизм внимания https://habr.com/ru/articles/458992/

In [ ]:
def _einsum(a, b, c, x, y):
    einsum_str = '{},{}->{}'.format(''.join(a), ''.join(b), ''.join(c))
    return torch.einsum(einsum_str, x, y)


def contract_inner(x, y):
    """tensordot(x, y, 1)."""
    x_chars = list(string.ascii_lowercase[:len(x.shape)])
    y_chars = list(string.ascii_lowercase[len(x.shape):len(y.shape) + len(x.shape)])
    y_chars[0] = x_chars[-1]  # first axis of y and last of x get summed
    out_chars = x_chars[:-1] + y_chars[1:]
    return _einsum(x_chars, y_chars, out_chars, x, y)

In [ ]:
class NIN(nn.Module):
    def __init__(self, in_dim, num_units, init_scale=0.1):
        super().__init__()
        self.W = nn.Parameter(default_init(scale=init_scale)((in_dim, num_units)), requires_grad=True)
        self.b = nn.Parameter(torch.zeros(num_units), requires_grad=True)

    def forward(self, x):
        x = x.permute(0, 2, 3, 1)
        y = contract_inner(x, self.W) + self.b
        return y.permute(0, 3, 1, 2)

In [ ]:
class AttnBlock(nn.Module):
    """Channel-wise self-attention block."""
    def __init__(self, channels):
        super().__init__()
        self.GroupNorm_0 = nn.GroupNorm(num_groups=32, num_channels=channels, eps=1e-6)
        self.NIN_0 = NIN(channels, channels)
        self.NIN_1 = NIN(channels, channels)
        self.NIN_2 = NIN(channels, channels)
        self.NIN_3 = NIN(channels, channels, init_scale=0.)

    def forward(self, x):
        B, C, H, W = x.shape
        h = self.GroupNorm_0(x)
        q = self.NIN_0(h)
        k = self.NIN_1(h)
        v = self.NIN_2(h)

        w = torch.einsum('bchw,bcij->bhwij', q, k) * (int(C) ** (-0.5))
        w = torch.reshape(w, (B, H, W, H * W))
        w = F.softmax(w, dim=-1)
        w = torch.reshape(w, (B, H, W, H, W))
        h = torch.einsum('bhwij,bcij->bchw', w, v)
        h = self.NIN_3(h)
        return x + h

### 5.7. Downsampling

Разберем процедуру $\textbf{Downsampling}$.

Во-первых,  стоит понимать, что процедура Downsapmling может быть как нейросетевой, так и детерминистической.

1. Детерминистическая процедура представляет собой применение  $\textbf{Average pooling}$ с размером ядра 2 и страйдом 2. Так мы усредняем значения в каждом квадрате 2 на 2, а значит, в 2 раза уменьшаем размер изображения.

2. Нейросетевая процедура. Здесь мы вместо $\textbf{Average pooling}$ заводим сверточную нейронную сеть с ядром  3 и страйдом 2. Таким образом, на выходе тоже получаем объект в 2 раза меньше исходного.

In [ ]:

class Downsample(nn.Module):
    def __init__(self, channels, with_conv=False):
        super().__init__()
        if with_conv:
            self.Conv_0 = ddpm_conv3x3(channels, channels, stride=2, padding=0)
        self.with_conv = with_conv

    def forward(self, x):
        B, C, H, W = x.shape
        # Emulate 'SAME' padding
        if self.with_conv:
            x = F.pad(x, (0, 1, 0, 1))
            x = self.Conv_0(x)
        else:
            x = F.avg_pool2d(x, kernel_size=2, stride=2, padding=0)

        assert x.shape == (B, C, H // 2, W // 2)
        return x

### 5.8. Upsampling

Класс Upsampling позволяет определить нейронно-сверточную процедуру:
    
- Если процедура происходит за счет несверточного метода, то размер изображения интерполируется в 2 раза.

- Если процедура сверточная: после того как мы растянули изображение в 2 раза, применяем базовую свертку 3 на 3.

**Вопрос**: что делает базовая свертка 3 на 3?


In [ ]:
class Upsample(nn.Module):
    def __init__(self, channels, with_conv=False):
        super().__init__()
        if with_conv:
            self.Conv_0 = ddpm_conv3x3(channels, channels)
        self.with_conv = with_conv

    def forward(self, x):
        B, C, H, W = x.shape
        h = F.interpolate(x, (H * 2, W * 2), mode='nearest')
        if self.with_conv:
            h = self.Conv_0(h)
        return h


### 5.9. ResNet Block

Теперь определим архитектуру сети, лежащей в основе диффузионной модели — ResNet, с встроенной в нее Group norm.

![title](https://www.baeldung.com/wp-content/uploads/sites/4/2024/02/group-normalization-and-other-approaches-1024x272.png)

In [ ]:
class ResnetBlockDDPM(nn.Module):
    """The ResNet Blocks used in DDPM."""
    def __init__(self, act, in_ch, out_ch=None, temb_dim=None, conv_shortcut=False, dropout=0.1):
        super().__init__()
        if out_ch is None:
            out_ch = in_ch
        self.GroupNorm_0 = nn.GroupNorm(num_groups=32, num_channels=in_ch, eps=1e-6)
        self.act = act
        self.Conv_0 = ddpm_conv3x3(in_ch, out_ch)
        if temb_dim is not None:
            self.Dense_0 = nn.Linear(temb_dim, out_ch)
            self.Dense_0.weight.data = default_init()(self.Dense_0.weight.data.shape)
            nn.init.zeros_(self.Dense_0.bias)

        self.GroupNorm_1 = nn.GroupNorm(num_groups=32, num_channels=out_ch, eps=1e-6)
        self.Dropout_0 = nn.Dropout(dropout)
        self.Conv_1 = ddpm_conv3x3(out_ch, out_ch, init_scale=0.)
        if in_ch != out_ch:
            if conv_shortcut:
                self.Conv_2 = ddpm_conv3x3(in_ch, out_ch)
            else:
                self.NIN_0 = NIN(in_ch, out_ch)
        self.out_ch = out_ch
        self.in_ch = in_ch
        self.conv_shortcut = conv_shortcut

    """
    def forward(self, x, temb=None):
        return checkpoint(self._forward, (x,) ,self.parameters(), use_checkpoint)
    """
    def forward(self, x, temb=None):
        B, C, H, W = x.shape
        assert C == self.in_ch
        out_ch = self.out_ch if self.out_ch else self.in_ch
        h = self.act(self.GroupNorm_0(x))
        h = self.Conv_0(h)
        # Add bias to each feature map conditioned on the time embedding
        if temb is not None:
            h += self.Dense_0(self.act(temb))[:, :, None, None]
        h = self.act(self.GroupNorm_1(h))
        h = self.Dropout_0(h)
        h = self.Conv_1(h)
        if C != out_ch:
            if self.conv_shortcut:
                x = self.Conv_2(x)
            else:
                x = self.NIN_0(x)
        return x + h

### 5.10 Model DDPM

In [ ]:
class DDPM(torch.nn.Module):

    def __init__(self, config):
        super().__init__()
        self.act = act = get_act(config)

        self.nf = nf = config.model.nf # dimensionality of time embedding
        self.conditional = conditional = config.model.conditional  # time conditional for diffusion  process

        modules = [] # This list is composed of nets
        if conditional:
            # Condition on noise levels.
            modules = [nn.Linear(nf, nf * 4)]
            modules[0].weight.data = default_init()(modules[0].weight.data.shape)
            nn.init.zeros_(modules[0].bias)
            modules.append(nn.Linear(nf * 4, nf * 4))
            modules[1].weight.data = default_init()(modules[1].weight.data.shape)
            nn.init.zeros_(modules[1].bias)

        self.centered = config.data.centered
        channels = config.data.num_channels


        # downsampling block #
        modules.append(ddpm_conv3x3(channels, nf))
        ch_mult = config.model.ch_mult
        dropout = config.model.dropout
        self.num_res_blocks = num_res_blocks = config.model.num_res_blocks
        self.num_resolutions = num_resolutions = len(ch_mult) # downsamples
        # all_resolutions: [16,8,4,2]
        self.all_resolutions = all_resolutions = [config.data.image_size // (2 ** i) for i in range(num_resolutions)]
        self.attn_resolutions = attn_resolutions = config.model.attn_resolutions
        resamp_with_conv = config.model.resamp_with_conv


        ResnetBlock = functools.partial(ResnetBlockDDPM, act=act, temb_dim=4 * nf, dropout=dropout)
        from   ContDDPM.models.ddpm_entities import AttnBlock
        AttnBlock = functools.partial(AttnBlock)

        #####################
        # Downsampling block#
        #####################
        hs_c = [nf]
        in_ch = nf

        for i_level in range(num_resolutions):

            # Residual blocks for this resolution
            for i_block in range(self.num_res_blocks):
                out_ch = nf * ch_mult[i_level]
                modules.append(ResnetBlock(in_ch=in_ch, out_ch=out_ch))
                in_ch = out_ch
                if all_resolutions[i_level] in attn_resolutions:
                    modules.append(AttnBlock(channels=in_ch))
                hs_c.append(in_ch)

            if i_level != num_resolutions - 1:
                modules.append(Downsample(channels=in_ch, with_conv=resamp_with_conv))
                hs_c.append(in_ch)

        in_ch = hs_c[-1]
        modules.append(ResnetBlock(in_ch=in_ch))
        modules.append(AttnBlock(channels=in_ch))
        modules.append(ResnetBlock(in_ch=in_ch))
        #####################


        #####################
        # Upsampling block#
        #####################
        for i_level in reversed(range(num_resolutions)):
            for i_block in range(num_res_blocks + 1):
                out_ch = nf * ch_mult[i_level]
                modules.append(ResnetBlock(in_ch=in_ch + hs_c.pop(), out_ch=out_ch))
                in_ch = out_ch
            if all_resolutions[i_level] in attn_resolutions:
                modules.append(AttnBlock(channels=in_ch))
            if i_level != 0:
                modules.append(Upsample(channels=in_ch, with_conv=resamp_with_conv))
        #####################


        assert not hs_c
        modules.append(nn.GroupNorm(num_channels=in_ch, num_groups=32, eps=1e-6))
        modules.append(ddpm_conv3x3(in_ch, channels, init_scale=0.))
        self.all_modules = nn.ModuleList(modules)


    def forward(self, x, labels):

        """
        Running Diffusion model


        inputs:
        - x: [B,C,H,W] = [B,3,16,16] images
        - labels: [B]                time

        returns:
        """

        modules = self.all_modules
        m_idx = 0


        ##################
        #Time embeddings #
        ##################
        if self.conditional:
            # timestep/scale embedding
            timesteps = labels # torch.Size([B])
            temb = get_timestep_embedding(timesteps, self.nf) # torch.Size([B, nf])
            # modules[0] = torch.nn.Linear(nf, nf*4)
            temb = modules[m_idx](temb) # torch.Size([B, nf*4])
            m_idx += 1
            # modules[1] = torch.nn.Linear(nf*4,nf*4) = torch.nn.Linear(512,512)
            temb = modules[m_idx](self.act(temb)) # torch.Size([B, nf*4])
            m_idx += 1
        else:
            temb = None
        #################



        ######################
        #Centering of Images #
        ######################
        if self.centered:
            # Input is in [-1, 1]
            h = x

            #assert torch.min(x).item() >= -1.1
            #assert torch.max(x).item() <=  1.1
        else:
            # Input is in [0, 1]
            h = 2 * x - 1.
            #assert torch.min(x).item() >= 0.1
            #assert torch.max(x).item() <= 1.1
        #################




        #####################
        # Downsampling block#
        #####################

        # torch.nn.Conv2D(3, nf, kernel_size=(3, 3), stride=1, padding=1)
        hs = [modules[m_idx](h)] # torch.Size([B,nf,16,16])
        m_idx += 1

        for i_level in range(self.num_resolutions): #self.num_resolutions

            # Residual blocks for this resolution

            #################################
            ##### while resolution = 16 #####
            ##################################


            # 1. ResNetBlock(in_ch=nf,out_ch = nf*1 = nf*ch_mult[0]) -> [torch.Size([B,nf,16,16])]
            # 2. AttnBlock -> [torch.Size([B,nf,16,16])]
            # 3. ResNetBlock(in_ch=nf,out_ch = nf*1 = nf*ch_mult[0]) -> [torch.Size([B,nf,16,16])]
            # 4. AttnBlock -> [torch.Size([B,nf,16,16])]
            # 5. ResNetBlock(in_ch=nf,out_ch = nf*1 = nf*ch_mult[0]) -> [torch.Size([B,nf,16,16])]
            # 6. AttnBlock -> [torch.Size([B,nf,16,16])]
            # 7. ResNetBlock(in_ch=nf,out_ch = nf*1 = nf*ch_mult[0]) -> [torch.Size([B,nf,16,16])]
            # 8. AttnBlock -> [torch.Size([B,nf,16,16])]
            # 9. DownSampling -> [torch.Size([B,nf,8,8])]
            # len(hs) = 6 : hs = [before, after_attn_1, ...., after_attn_4, after_downsampling]

            ##################################


            #################################
            ##### while resolution = 8 #####
            ##################################



            #################################


            #4 times ResNet(in_ch=nf,out_ch=nf), 4 times hs.append(torch.Size([B,nf,16,16]))
            # Downsample step: hs.append([torch.Size([B,nf,8,8])])

            # while resolution = 8: ResNet(in_ch=nf,out_ch=nf*2) and Attn(nf*2) , 4 times hs.append(torch.Size([B,nf*2,8,8]))
            # Downsample step: hs.append([torch.Size([B,nf*2,4,4])])

            # while resolution = 4: 4 times ResNet(in_ch=nf*2,out_ch=nf*2), 4 times hs.append(torch.Size([B,nf*2,4,4]))
            # Downsample step: hs.append([torch.Size([B,nf*2,2,2])])

            # while resolution = 2: 4 times ResNet(in_ch=nf*2,out_ch=nf*2), 4 times hs.append(torch.Size([B,nf*2,2,2]))
            # without last Downsample step

            for i_block in range(self.num_res_blocks):
                h = modules[m_idx](hs[-1], temb)

                m_idx += 1
                if h.shape[-1] in self.attn_resolutions:
                    # Application of Attention block
                    h = modules[m_idx](h)
                    m_idx += 1
                hs.append(h)

            if i_level != self.num_resolutions - 1:
                # Application of DownSample block
                hs.append(modules[m_idx](hs[-1]))
                m_idx += 1

        # hs[-1] = torch.Size([B,nf*2,2,2])
        # temb  = torch.Size([B,nf*4])


        h = hs[-1]
        h = modules[m_idx](h, temb) # torch.Size([B,nf*2,2,2])
        m_idx += 1
        h = modules[m_idx](h) # torch.Size([B,nf*2,2,2])
        m_idx += 1
        h = modules[m_idx](h, temb)# torch.Size([B,nf*2,2,2])
        m_idx += 1
        #####################




        #####################
        # Upsampling block#
        #####################
        for i_level in reversed(range(self.num_resolutions)):
            for i_block in range(self.num_res_blocks + 1):
                h = modules[m_idx](torch.cat([h, hs.pop()], dim=1), temb)
                m_idx += 1

            if h.shape[-1] in self.attn_resolutions:
                h = modules[m_idx](h)
                m_idx += 1
            if i_level != 0:
                h = modules[m_idx](h)
                m_idx += 1

        assert not hs
        h = self.act(modules[m_idx](h))
        m_idx += 1
        h = modules[m_idx](h)
        m_idx += 1
        assert m_idx == len(modules)

        return h


In [ ]:
config = # создайте ваш конфиг
config.data.image_size=16
model = DDPM(config )

In [ ]:
model.modules

<bound method Module.modules of DDPM(
  (act): SiLU()
  (all_modules): ModuleList(
    (0): Linear(in_features=128, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=512, bias=True)
    (2): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ResnetBlockDDPM(
      (GroupNorm_0): GroupNorm(32, 128, eps=1e-06, affine=True)
      (act): SiLU()
      (Conv_0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (Dense_0): Linear(in_features=512, out_features=128, bias=True)
      (GroupNorm_1): GroupNorm(32, 128, eps=1e-06, affine=True)
      (Dropout_0): Dropout(p=0.1, inplace=False)
      (Conv_1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (4): AttnBlock(
      (GroupNorm_0): GroupNorm(32, 128, eps=1e-06, affine=True)
      (NIN_0): NIN()
      (NIN_1): NIN()
      (NIN_2): NIN()
      (NIN_3): NIN()
    )
    (5): ResnetBlockDDPM(
      (GroupNorm_0): GroupNorm(32, 128, eps=1e-06, 

## 6. Данные

Данная функция берет картинки из датасета MNIST. Раскрасим цифры разными цветами, созранив темный бэкграунд:

![title](https://iimg.su/s/18/XP2QCS85msDHHaIf7TFJvfxf3wt7htnWNqF4EyJR.png)

In [ ]:
def get_random_colored_images(images, seed = 0x000000):
    np.random.seed(seed)

    images = 0.5*(images + 1)
    size = images.shape[0]
    colored_images = []
    hues = 360*np.random.rand(size)

    for V, H in zip(images, hues):
        V_min = 0

        a = (V - V_min)*(H%60)/60
        V_inc = a
        V_dec = V - a

        colored_image = torch.zeros((3, V.shape[1], V.shape[2]))
        H_i = round(H/60) % 6

        if H_i == 0:
            colored_image[0] = V
            colored_image[1] = V_inc
            colored_image[2] = V_min
        elif H_i == 1:
            colored_image[0] = V_dec
            colored_image[1] = V
            colored_image[2] = V_min
        elif H_i == 2:
            colored_image[0] = V_min
            colored_image[1] = V
            colored_image[2] = V_inc
        elif H_i == 3:
            colored_image[0] = V_min
            colored_image[1] = V_dec
            colored_image[2] = V
        elif H_i == 4:
            colored_image[0] = V_inc
            colored_image[1] = V_min
            colored_image[2] = V
        elif H_i == 5:
            colored_image[0] = V
            colored_image[1] = V_min
            colored_image[2] = V_dec

        colored_images.append(colored_image)

    colored_images = torch.stack(colored_images, dim = 0)
    colored_images = 2*colored_images - 1

    return colored_images

Следующая функция создает генератор для наших данных, батчи из которого мы будем использовать во время обучения.

In [ ]:
class DataGenerator:
    def __init__(self, config):
        self.config = config

        if self.config.data.dataset.startswith('shoes'):
            test_ratio=0.1
            dataset = h5py_to_dataset(config.data.path, config.data.image_size)
            idx = list(range(len(dataset)))
            test_size = int(len(idx) * test_ratio)
            train_idx, test_idx = idx[:-test_size], idx[-test_size:]
            train_set, test_set = Subset(dataset, train_idx), Subset(dataset, test_idx)

            self.train_loader = DataLoader(train_set, shuffle=True, num_workers=0, batch_size=config.data.batch_size)
            self.valid_loader = DataLoader(test_set, shuffle=True, num_workers=0, batch_size=config.data.batch_size)

        elif self.config.data.dataset.startswith('celeba_male'):

            test_ratio=0.1
            transform = Compose([ CenterCrop(140),
                                  Resize((config.data.image_size, config.data.image_size)),
                                  ToTensor(),
                                  #Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                  #Lambda(lambda x: (x+1)/2 )]
                                ])

            dataset = ImageFolder(config.data.path, transform=transform)
            idx = list(range(len(dataset)))
            test_size = int(len(idx) * test_ratio)
            train_idx, test_idx = idx[:-test_size], idx[-test_size:]
            train_set, test_set = Subset(dataset, train_idx), Subset(dataset, test_idx)

            self.train_loader = DataLoader(train_set, shuffle=True, num_workers=0, batch_size=config.data.batch_size)
            self.valid_loader = DataLoader(test_set, shuffle=True, num_workers=0, batch_size=config.data.batch_size)

        elif self.config.data.dataset.startswith('anime'):
            test_ratio=0.1
            transform = Compose([Resize((config.data.image_size, config.data.image_size)),
                                 ToTensor(),
                                 #Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                 ])
            dataset = ImageFolder(config.data.path, transform=transform)
            idx = list(range(len(dataset)))
            test_size = int(len(idx) * test_ratio)
            train_idx, test_idx = idx[:-test_size], idx[-test_size:]
            train_set, test_set = Subset(dataset, train_idx), Subset(dataset, test_idx)

            self.train_loader = DataLoader(train_set, shuffle=True, num_workers=0, batch_size=config.data.batch_size)
            self.valid_loader = DataLoader(test_set, shuffle=True, num_workers=0, batch_size=config.data.batch_size)

        elif self.config.data.dataset.startswith('church'):
            test_ratio=0.3
            dataset = np.load(self.config.data.path)



            idx = list(range(len(dataset)))
            test_size = int(len(idx) * test_ratio)
            train_idx, test_idx = idx[:-test_size], idx[-test_size:]

            train_set = 2 * (torch.tensor(np.array(dataset[train_idx]), dtype=torch.float32) / 255.).permute(0, 3, 1, 2) - 1
            train_set = F.interpolate(train_set,config.data.image_size , mode='bilinear')

            test_set = 2 * (torch.tensor(np.array(dataset[test_idx]), dtype=torch.float32) / 255.).permute(0, 3, 1, 2) - 1
            test_set = F.interpolate(test_set,config.data.image_size , mode='bilinear')

            train_set, test_set = Subset(train_set, train_idx), Subset(test_set, test_idx)

            self.train_loader = DataLoader(train_set, shuffle=True, num_workers=0, batch_size=config.data.batch_size)
            self.valid_loader = DataLoader(test_set, shuffle=True, num_workers=0, batch_size=config.data.batch_size)

        else:
            self.train_mnist_transforms = Compose(
                [
                    # Resize((config.data.image_size, config.data.image_size)),
                    Resize((config.data.image_size, config.data.image_size)),
                    ToTensor(),
                    #Normalize(mean=config.data.norm_mean, std=config.data.norm_std),
                    # to [-1; 1]
                    Lambda(lambda x:  2*x-1 )
                ]
            )


            self.valid_mnist_transforms = Compose(
                [
                    # Resize((config.data.image_size, config.data.image_size)),
                    Resize((config.data.image_size, config.data.image_size)),
                    ToTensor(),
                    #Normalize(mean=config.data.norm_mean, std=config.data.norm_std),
                    # to [-1; 1]
                    Lambda(lambda x: 2*x-1 )
                ]
            )

            dataset_name = config.data.dataset.split("_")[0]
            is_colored = dataset_name[-7:] == "colored"
            classes = [int(number) for number in config.data.dataset.split("_")[1:]]
            if not classes:
                classes = [i for i in range(10)]

            train_set =  MNIST(config.data.path, train=True, transform=self.train_mnist_transforms, download=True)
            test_set =  MNIST(config.data.path, train=False, transform=self.valid_mnist_transforms, download=True)

            train_test = []
            for dataset in [train_set, test_set]:
                data = []
                labels = []
                for k in range(len(classes)):
                    data.append(torch.stack(
                        [dataset[i][0] for i in range(len(dataset.targets)) if dataset.targets[i] == classes[k]],
                        dim=0
                    ))
                    labels += [k]*data[-1].shape[0]
                data = torch.cat(data, dim=0)
                data = data.reshape(-1, 1, config.data.image_size, config.data.image_size)
                labels = torch.tensor(labels)

                if is_colored:
                    data = get_random_colored_images(data)

                train_test.append(TensorDataset(data, labels))

            train_set, test_set = train_test



            self.train_loader = DataLoader(
                train_set,
                batch_size=config.training.batch_size,
                shuffle=True,
                drop_last=True
            )


            self.valid_loader = DataLoader(
                test_set,
                batch_size= config.training.batch_size,
                shuffle=False,
                drop_last=False
            )

    def sample_train(self):
        while True:
            for batch in self.train_loader:
                yield batch


## 7. Diffusion Runner

![title](https://habrastorage.org/getpro/habr/upload_files/de1/754/3a4/de17543a4f693c78f50f6f93b231374b.png)


Пайплайн модели:

- Семплируем батч данных
- Выбираем различные уровни зашумления
- Зашумляем наши данные из батча
- По зашумленным точкам вычисляем score-функцию $\nabla \log p(x)$
- Запускаем обратный процесс семплирования


In [ ]:
class DiffusionRunner:
    def __init__(
            self,
            config: ConfigDict,
            eval: bool = False
    ):
        self.config = config
        self.eval = eval

        self.model = DDPM(config=config)
        self.sde = VP_SDE(config=config)
        self.rsde = RSDE(self.sde, ode_sampling=config.training.ode_sampling)
        self.diff_eq_solver = EulerDiffEqSolver(self.sde, self.rsde,
                                                self.calc_score,
                                                ode_sampling=config.training.ode_sampling)

        #self.inverse_scaler = lambda x: torch.clip(127.5 * (x + 1), 0, 255)
        self.inverse_scaler = lambda x: torch.clip( 255 * x, 0, 255)

        self.checkpoints_folder = config.training.checkpoints_folder
        if eval:
            self.ema = ExponentialMovingAverage(self.model.parameters(), config.model.ema_rate)
            self.restore_parameters()
            self.switch_to_ema()

        device = torch.device(self.config.device)
        self.device = device
        self.model.to(device)

    def restore_parameters(self, device: Optional[torch.device] = None) -> None:
        checkpoints_folder: str = self.checkpoints_folder
        if device is None:
            device = torch.device('cpu')
        model_ckpt = torch.load(checkpoints_folder + '/model.pth', map_location=device)
        self.model.load_state_dict(model_ckpt)

        ema_ckpt = torch.load(checkpoints_folder + '/ema.pth', map_location=device)
        self.ema.load_state_dict(ema_ckpt)

    def switch_to_ema(self) -> None:
        ema = self.ema
        score_model = self.model
        ema.store(score_model.parameters())
        ema.copy_to(score_model.parameters())

    def switch_back_from_ema(self) -> None:
        ema = self.ema
        score_model = self.model
        ema.restore(score_model.parameters())

    def set_optimizer(self) -> None:
        optimizer = torch.optim.Adam(
            self.model.parameters(),
            lr=self.config.optim.lr,
            betas=(0.9, 0.999),
            eps=1e-8,
            weight_decay=self.config.optim.weight_decay
        )
        self.warmup = self.config.optim.linear_warmup
        self.grad_clip_norm = self.config.optim.grad_clip_norm
        self.optimizer = optimizer

    def calc_score(self, input_x: torch.Tensor, input_t: torch.Tensor, y=None) -> torch.Tensor:
        """
        calculate score w.r.t noisy X and t
        """
        model_output = self.model(input_x, input_t)
        curr_std = self.sde.marginal_std(input_t)

        score = - model_output / curr_std[:, None, None, None]
        return score

    def sample_time(self, batch_size: int, eps: float = 1e-5):
        return torch.rand(batch_size) * (self.sde.T - eps) + eps

    def calc_loss(self, clean_x: torch.Tensor, eps: float = 1e-5) -> Union[float, torch.Tensor]:
        """
        Define score-matching MSE loss
        """
        # здесь сэмплируем время, прогоняем через сетку и рассчитываем MSE-лосс

        batch_size = clean_x.shape[0]
        t = self.sample_time(batch_size, eps).to(self.device) # sample time

        noise = self.sde.prior_sampling(clean_x.shape).to(self.device)  # noise batch from N(0, I)

        # forward_pass + зашумление
        mean, std = self.sde.marginal_prob(clean_x, t)
        noised_x = mean + std[:, None, None, None] * noise

        # score_estimation

        rebuilt_noise = self.model(noised_x, t)

        # сalc_loss
        loss = F.mse_loss(rebuilt_noise, noise)

        return loss

    def set_data_generator(self) -> None:
        self.datagen = DataGenerator(self.config)

    def manage_optimizer(self) -> None:
        self.lrs = []
        if self.warmup > 0 and self.step < self.warmup:
            for g in self.optimizer.param_groups:
                self.lrs += [g['lr']]
                g['lr'] = g['lr'] * float(self.step + 1) / self.warmup
        if self.grad_clip_norm is not None:
            torch.nn.utils.clip_grad_norm_(
                self.model.parameters(),
                max_norm=self.grad_clip_norm
            )

    def restore_optimizer_state(self) -> None:
        if self.lrs:
            self.lrs = self.lrs[::-1]
            for g in self.optimizer.param_groups:
                g['lr'] = self.lrs.pop()

    def log_metric(self, metric_name: str, loader_name: str, value: Union[float, torch.Tensor, wandb.Image]):
        wandb.log({f'{metric_name}/{loader_name}': value}, step=self.step)

    def optimizer_step(self, loss: torch.Tensor) -> None:
        self.optimizer.zero_grad()
        loss.backward()

        self.manage_optimizer()
        self.log_metric('lr', 'train', self.optimizer.param_groups[0]['lr'])
        self.optimizer.step()
        self.ema.update(self.model.parameters())
        self.restore_optimizer_state()

    def validate(self) -> None:
        prev_mode= self.model.training

        self.model.eval()
        self.switch_to_ema()

        valid_loss = 0
        valid_count = 0
        with torch.no_grad():
            for (X,y) in self.datagen.valid_loader:
                X = X.to(self.device)
                loss = self.calc_loss(clean_x=X)
                valid_loss += loss.item() * X.size(0)
                valid_count += X.size(0)

        valid_loss = valid_loss / valid_count
        self.log_metric('loss', 'valid_loader', valid_loss)

        self.switch_back_from_ema()
        self.model.train(prev_mode)

    def train(self) -> None:
        self.set_optimizer()
        self.set_data_generator()
        train_generator = self.datagen.sample_train()
        self.step = 0

        wandb.init(project='sde', name='ddpm_cont')

        self.ema = ExponentialMovingAverage(self.model.parameters(), self.config.model.ema_rate)
        self.model.train()
        for iter_idx in trange(1, 1 + self.config.training.training_iters):
            self.step = iter_idx

            if self.config.data.dataset == "church":
                X =  next(train_generator)
            else:
                (X, y) = next(train_generator)

            #print(torch.min(X[0]), torch.max(X[0]) )
            X = X.to(self.device)
            loss = self.calc_loss(clean_x=X)
            self.log_metric('loss', 'train', loss.item())
            self.optimizer_step(loss)

            if iter_idx % self.config.training.snapshot_freq == 0:
                self.snapshot()

            if iter_idx % self.config.training.eval_freq == 0:
                self.validate()

            if iter_idx % self.config.training.checkpoint_freq == 0:
                self.save_checkpoint()

        self.model.eval()
        self.save_checkpoint()
        self.switch_to_ema()

    def save_checkpoint(self) -> None:
        if not os.path.exists(self.checkpoints_folder):
            os.makedirs(self.checkpoints_folder)
        torch.save(self.model.state_dict(), os.path.join(self.checkpoints_folder,
                                                               f'model.pth'))
        torch.save(self.ema.state_dict(), os.path.join(self.checkpoints_folder,
                                                       f'ema.pth'))
        torch.save(self.optimizer.state_dict(), os.path.join(self.checkpoints_folder,
                                                             f'opt.pth'))

    def reset_unconditional_sampling() -> None:
        self.diff_eq_solver = EulerDiffEqSolver(
            self.sde,
            self.calc_score,
            self.config.training.ode_sampling
        )

    def set_conditional_sampling(
            self,
            classifier_grad_fn: Callable[["NoisyImages", "T", "Labels"], "Scores"],
            T: float = 1.0
    ) -> None:
        def new_score_fn(x, t, y):
            """
            define posterior_score w.r.t T
            """
            return posterior_score_T

        self.diff_eq_solver = EulerDiffEqSolver(
            self.sde,
            new_score_fn,
            self.config.training.ode_sampling
        )


    def set_classifier(self, classifier: torch.nn.Module, T: float = 1.0) -> None:
        self.classifier = classifier
        def classifier_grad_fn(x, t, y):
            """
            calculate likelihood_score with torch.autograd.grad
            """
            return likelihood_score

        self.set_conditional_sampling(classifier_grad_fn, T=T)


    def sample_images(
            self, batch_size: int,
            eps:float = 1e-5,
            labels: Optional[torch.Tensor] = None
    ) -> torch.Tensor:
        shape = (
            batch_size,
            self.config.data.num_channels,
            self.config.data.image_size,
            self.config.data.image_size
        )
        device = torch.device(self.config.device)
        with torch.no_grad():
            """
            Implement cycle for Euler RSDE sampling w.r.t labels
            labels = None if uncond. gen is used
            """

            # сэмплируем нормальный шум
            pred_images = self.sde.prior_sampling(shape).to(self.device)
            # делаем временную сетку размера N
            timesteps = torch.linspace(self.sde.T, eps, self.sde.N, device=self.device)
            # применяем метод Эйлера
            for i in range(self.sde.N):
                # делаем батч для времени, устанавливая везде текущее время
                time = torch.ones(batch_size, device=self.device) * timesteps[i]
                # делаем шаг метода Эйлера
                pred_images, _ = self.diff_eq_solver.step(pred_images, time, y=labels)

        return self.inverse_scaler(pred_images)


    def snapshot(self, labels: Optional[torch.Tensor] = None) -> None:
        prev_mode = self.model.training

        self.model.eval()
        self.switch_to_ema()

        images = self.sample_images(self.config.training.snapshot_batch_size, labels=labels).cpu()
        nrow = int(math.sqrt(self.config.training.snapshot_batch_size))
        grid = torchvision.utils.make_grid(images, nrow=nrow).permute(1, 2, 0)
        grid = grid.data.numpy().astype(np.uint8)
        self.log_metric('images', 'from_noise', wandb.Image(grid))

        self.switch_back_from_ema()
        self.model.train(prev_mode)

    def train_classifier(
            self,
            classifier: torch.nn.Module,
            classifier_optim: torch.optim.Optimizer,
            classifier_loss: Callable[[torch.Tensor, torch.Tensor], torch.Tensor],
            T: float = 10.0
    ) -> None:
        device = torch.device(self.config.device)
        self.device = device

        self.set_classifier(classifier, T=T)

        self.step = 0

        wandb.init(project='sde', name='noisy_classifier')

        def get_logits(X, y):
            t = self.sample_time(X.size(0)).to(device)

            """calc logits"""

            return loss, pred_labels

        self.set_data_generator()
        train_generator = self.datagen.sample_train()
        classifier.train()

        self.config.training.snapshot_batch_size = 100
        labels = np.tile(np.arange(10), (10, 1))
        labels = torch.Tensor(labels).to(device).long().view(-1)

        for iter_idx in trange(1, 1 + self.config.classifier.training_iters):
            self.step = iter_idx

            """
            train classifier
            """

            if iter_idx % self.config.classifier.snapshot_freq == 0:
                self.snapshot(labels=labels)

            if iter_idx % self.config.classifier.eval_freq == 0:
                valid_loss = 0
                valid_accuracy = 0
                valid_count = 0
                classifier.eval()
                with torch.no_grad():
                    """
                    validate classifier
                    """
                valid_loss = valid_loss / valid_count
                valid_accuracy = valid_accuracy / valid_count
                self.log_metric('cross_entropy', 'valid', valid_loss)
                self.log_metric('accuracy', 'valid', valid_accuracy)
                classifier.train()

            if iter_idx % self.config.classifier.checkpoint_freq == 0:
                torch.save(
                    classifier.state_dict(),
                    self.config.classifier.checkpoint_path
                )

        classifier.eval()
        torch.save(
            classifier.state_dict(),
            self.config.classifier.checkpoint_path
        )

In [ ]:
config =  create_default_cm_2_config()


config.data.dataset = "MNIST-colored_2"
config.data.image_size = 16
config.data.num_channels = 3
config.data.centered = True
config.data.batch_size = 32
config.data.norm_mean = (0.5)
config.data.norm_std = (0.5)
config.data.path = "/trinity/home/a.kolesov/data/MNIST"

# model
config.model.ch_mult = (1, 2, 2, 2)
config.model.num_res_blocks = 4
config.model.attn_resolutions = (16,)
config.model.dropout = 0.1
config.model.resamp_with_conv = True
config.model.conditional = True
config.model.nonlinearity = 'swish'
diffusion = DiffusionRunner(config)

In [ ]:
diffusion.train()

loss/train,0.01566
lr/train,0.0002
_runtime,1484
_timestamp,1728484964
_step,5000


loss/train,█▇▅▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr/train,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇██████████
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


wandb: wandb version 0.18.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


  0%|          | 0/80000 [00:00<?, ?it/s]

**Полезные ссылки:**

- Репозиторий метода https://notebooks.githubusercontent.com/view/ipynb?browser=unknown_browser&color_mode=auto&commit=73bb1b5bd6c7af6b06abc52957de76efd4e91175&device=unknown_device&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f6d727961622f646c2d6873652d616d692f373362623162356264366337616636623036616263353239353764653736656664346539313137352f7765656b31305f70726f626d6f64656c732f686f6d65776f726b2e6970796e62&logged_in=false&nwo=mryab%2Fdl-hse-ami&path=week10_probmodels%2Fhomework.ipynb&platform=unknown_platform&repository_id=531107718&repository_type=Repository&version=0


- Блог с объяснениями непрерывного метода https://allanchan339.github.io/2022/12/22/Review-DDIM/

**Заключение**:

1. Познакомились с теорией СДУ
2. Разобрались с понятием маргинального распределения
3. Вывели уравнения моментов СДУ
4. Разобрали пайплайн непрывной диффузионной модели